In [1]:
import numpy as np

modelnames = ['Layered Connectivity\n(SCB)',
              'Layered Connectivity\n(XT3D)',
              'Full Connectivity\n(SCB)',
              'Full Connectivity\n(XT3D)']

In [ ]:
def fig2(i, vmin, vmax, extent, scale_chan, scale_bndy, scale_dom, normalize):    
    
    fig2cell(fig, i, vmin, vmax, extent, scale_chan, scale_dom, normalize)
    fig2face(fig, i, vmin, vmax, extent, scale_chan, scale_bndy, scale_dom, normalize)
    fig2head(fig, i, vmin, vmax, extent, scale_chan)

In [ ]:
def fig2cell(fig, i, vmin, vmax, extent, scale_chan, scale_dom, normalize):    

    qx_chan = np.ma.masked_where(icelltype != 1, qx)
    qy_chan = np.ma.masked_where(icelltype != 1, qy)
    qz_chan = np.ma.masked_where(icelltype != 1, qz)
    qx_dom = np.ma.masked_where(icelltype != 0, qx)
    qy_dom = np.ma.masked_where(icelltype != 0, qy)
    qz_dom = np.ma.masked_where(icelltype != 0, qz)
    j = i * 3 + 1
    ax = plt.subplot(4,3,j,aspect="equal")
    xs = flopy.plot.PlotCrossSection(model=gwf, line={"line": [(0, 0.5),(Lx, 0.5)]}, extent=extent,)
    cbct = xs.plot_array(a = icelltype, cmap = 'gray', alpha=0.4, vmin = -4, vmax = 1)
    xs.plot_vector(qx_chan, qy_chan, qz_chan, scale=scale_chan, normalize=normalize, color="black")
    xs.plot_vector(qx_dom, qy_dom, qz_dom, scale=scale_dom, normalize=normalize, color="gray")
    xs.plot_grid(lw = 0.1, color = 'black')
    if j < 10: ax.set_xticks([])
    if j == 11: ax.set_xlabel('x (m)', size = 12)
    ax.set_ylabel('z (m)', size = 12) 
    ax.annotate(modelnames[i], (0.5,10.7), size = 10, bbox=dict(facecolor = 'white'))


In [ ]:
def fig2face(fig, i, vmin, vmax, extent, scale_chan, scale_bndy, scale_dom, normalize):
        
    gp = d2d.get_gridprops_disu6()
    nja = gp["nja"]
    ihc = gp["ihc"]

    qxface = np.zeros((nja))
    qzface = np.zeros((nja))
    for iconn in range(nja):
        if ihc[iconn] > 0:
            qxface[iconn] = qnface[iconn]
        else:
            qzface[iconn] = qnface[iconn]
    j = i * 3 + 2
    ax = plt.subplot(4,3,j,aspect="equal")
    #ax.set_title("face fluxes", size = 12)
    xs = flopy.plot.PlotCrossSection(model=gwf, line={"line": [(0, 0.5),(Lx, 0.5)]}, extent=extent,)
    cbct = xs.plot_array(a = icelltype, cmap = 'gray', alpha=0.4, vmin = -4, vmax = 1)
    #cb = xs.contour_array(a = head, cmap='Purples', alpha=0.4, vmin = vmin, vmax = vmax)
    qxplot = np.ma.masked_where(ifacetype != 0, qxface)
    qzplot = np.ma.masked_where(ifacetype != 0, qzface)
    plt.quiver(xface, zface, qxplot, qzplot, scale=scale_dom, pivot='mid', color='blue')
    qxplot = np.ma.masked_where(ifacetype != 1, qxface)
    qzplot = np.ma.masked_where(ifacetype != 1, qzface)
    plt.quiver(xface, zface, qxplot, qzplot, scale=scale_bndy, pivot='mid', color='green')
    qxplot = np.ma.masked_where(ifacetype != 2, qxface)
    qzplot = np.ma.masked_where(ifacetype != 2, qzface)
    plt.quiver(xface, zface, qxplot, qzplot, scale=scale_chan, pivot='mid', color='red')
    xs.plot_grid(lw = 0.1, color = 'gray')
    ax.set_yticks([])
    if j < 10: ax.set_xticks([])
    if j == 11: ax.set_xlabel('x (m)', size = 12)
    ax.annotate("Flux Magnitude = %0.2f \nFlux direction = %0.2f\nFlow error = %0.2f" 
               %(q_all[i][0], q_all[i][1], q_all[i][2]),
                (0.5,10.4), size = 9, bbox=dict(facecolor = 'white'))


In [ ]:
def fig2head(fig, i, vmin, vmax, extent, scale_chan):
    
    import matplotlib.colors
    cmap1 = matplotlib.colors.LinearSegmentedColormap.from_list("", ["blue","blue","blue"])
    cmap2 = matplotlib.colors.LinearSegmentedColormap.from_list("", ["black","black","black"])
    
    j = i * 3 + 3
    ax = plt.subplot(4,3,j,aspect="equal")
    xs = flopy.plot.PlotCrossSection(model=gwf, line={"line": [(0, 0.5),(Lx, 0.5)]}, extent=extent,)
    cbct = xs.plot_array(a = icelltype, cmap = 'gray', alpha=0.4, vmin = -4, vmax = 1)
    cb = xs.contour_array(a = head,        cmap=cmap1, alpha=0.8, vmin = vmin, vmax = vmax)
    cb = xs.contour_array(a = head_analyt, cmap=cmap2, alpha=0.8, vmin = vmin, vmax = vmax)
    xs.plot_grid(lw = 0.1, color = 'black')
    ax.set_yticks([])
    if j < 10: ax.set_xticks([])
    if j == 11: ax.set_xlabel('x (m)', size = 12)

In [ ]:
def fig3():
       
    fig, axes = plt.subplots(3,2, figsize=(8, 6), constrained_layout=True)
   
    # FLUX MAGNITUDE
    ax = plt.subplot(321)
    ax.set_title('Layered Connectivity\n', size = 12)
    ax.set_ylim([0.5,1.2])
    ax.set_ylabel('Flux magnitude\n(m/d)\n', size = 10)
    ax.plot(resolution_scenarios, fig3_results[:,0,0], marker = 'o', linestyle = '-') # LC - SCB
    ax.plot(resolution_scenarios, fig3_results[:,1,0], marker = 'o', linestyle = '-') # FC - SCB
    ax.plot(resolution_scenarios, np.ones(len(resolution_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks([])
    ax.legend(['SCB', 'XT3D', 'Analytical'], loc = 'lower right')
    
    ax = plt.subplot(322)
    ax.set_title('Full Connectivity\n', size = 12)
    ax.set_ylim([0.5,1.2])
    ax.plot(resolution_scenarios, fig3_results[:,2,0], marker = 'o', linestyle = '-') # LC - XT3D
    ax.plot(resolution_scenarios, fig3_results[:,3,0], marker = 'o', linestyle = '-') # FC - XT3D
    ax.plot(resolution_scenarios, np.ones(len(resolution_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks([])
    ax.set_yticks([])
    ax.legend(['SCB', 'XT3D', 'Analytical'], loc = 'lower right')
    
    ax = plt.subplot(323)
    ax.set_ylim([-5,35])
    ax.set_ylabel('Flux angle\n(degrees)\n', size = 10)
    ax.plot(resolution_scenarios, fig3_results[:,0,1], marker = 'o', linestyle = '-') # LC - SCB
    ax.plot(resolution_scenarios, fig3_results[:,1,1], marker = 'o', linestyle = '-') # FC - SCB
    ax.plot(resolution_scenarios, 30*np.ones(len(resolution_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks([])
    
    ax = plt.subplot(324)
    ax.set_ylim([-5,35])
    ax.plot(resolution_scenarios, fig3_results[:,2,1], marker = 'o', linestyle = '-') # LC - XT3D
    ax.plot(resolution_scenarios, fig3_results[:,3,1], marker = 'o', linestyle = '-') # FC - XT3D
    ax.plot(resolution_scenarios, 30*np.ones(len(resolution_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks([])
    ax.set_yticks([])
    
    ax = plt.subplot(325)
    ax.set_xlabel('\nModel layers within channel', size = 10)
    ax.set_ylabel('Flow error', size = 10)
    ax.plot(resolution_scenarios, fig3_results[:,0,2], marker = 'o', linestyle = '-') # LC - SCB
    ax.plot(resolution_scenarios, fig3_results[:,1,2], marker = 'o', linestyle = '-') # FC - SCB
    ax.plot(resolution_scenarios, np.zeros(len(resolution_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks(resolution_scenarios)
    ax.set_ylim([-0.2,0.2])
    
    ax = plt.subplot(326)
    ax.set_xlabel('\nModel layers within channel', size = 10)
    ax.plot(resolution_scenarios, fig3_results[:,2,2], marker = 'o', linestyle = '-') # LC - XT3D
    ax.plot(resolution_scenarios, fig3_results[:,3,2], marker = 'o', linestyle = '-') # FC - XT3D
    ax.plot(resolution_scenarios, np.zeros(len(resolution_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks(resolution_scenarios)
    ax.set_yticks([])
    ax.set_ylim([-0.2,0.2])

    plt.tight_layout()
    plt.show()
    fig.savefig('../figures/fig3paper.png', dpi = 450)

In [3]:
def fig4():    
    
    x = dip
    #logfunc = lambda e: np.log10(e)
    #y = logfunc(Kcontrast)
    
    for s in range(4): # Four scenarios
               
        fig, axs = plt.subplots(1, 2, figsize = (8,2))
        fig.suptitle = modelnames[s]
        
        # FLUX MAG
        ax = plt.subplot(121)
        for kc, kcval in enumerate(Kcontrast):
            z = qmag_results[s][kc]
            ax.plot(x, z, marker = None, linestyle = '-', label = kcval)
            ax.set_title('Flux magnitude (m/d)')
            ax.set_xlabel('Dip (degrees)')
            ax.set_xticks([])
            ax.legend(loc = 'upper left')
            
        ax.plot(x, np.ones(len(x)), linestyle = '--', color = 'black') # Analytical
        ax.set_ylim([0,3])

        # FLUX ANGLE
        ax = plt.subplot(122)
        for kc, kcval in enumerate(Kcontrast):
            z = qang_results[s][kc]
            ax.plot(x, z, marker = None, linestyle = '-', label = kcval)
            ax.set_title('Flux direction (degrees)')
            ax.set_xlabel('Dip (degrees)')
            ax.set_xticks([])
            ax.legend(loc = 'upper left')
        ax.plot(x, x, linestyle = '--', color = 'black') # Analytical
        ax.set_ylim([0,100])
        
        plt.tight_layout()
        plt.show()
        fig.savefig('../figures/fig4_%i_paper.png' %s, dpi = 450)

In [ ]:
def fig5():
       
    fig, axes = plt.subplots(3,2, figsize=(8, 6), constrained_layout=True)
   
    # FLUX MAGNITUDE
    ax = plt.subplot(321)
    ax.set_title('Layered Connectivity\n', size = 12)
    ax.set_ylabel('Flux magnitude\n(m/d)\n', size = 10)
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,0,0], marker = 'o', linestyle = '-') # LC - SCB
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,1,0], marker = 'o', linestyle = '-') # FC - SCB
    ax.plot(np.log10(anisotropic_scenarios), np.ones(len(anisotropic_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks([])
    
    ax = plt.subplot(322)
    ax.set_title('Full Connectivity\n', size = 12)
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,2,0], marker = 'o', linestyle = '-') # LC - XT3D
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,3,0], marker = 'o', linestyle = '-') # FC - XT3D
    ax.plot(np.log10(anisotropic_scenarios), np.ones(len(anisotropic_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks([])
    ax.set_yticks([])
    
    ax = plt.subplot(323)
    ax.set_ylabel('Flux angle\n(degrees)\n', size = 10)
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,0,1], marker = 'o', linestyle = '-') # LC - SCB
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,1,1], marker = 'o', linestyle = '-') # FC - SCB
    ax.plot(np.log10(anisotropic_scenarios), 30*np.ones(len(anisotropic_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks([])
    
    ax = plt.subplot(324)
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,2,1], marker = 'o', linestyle = '-') # LC - XT3D
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,3,1], marker = 'o', linestyle = '-') # FC - XT3D
    ax.plot(np.log10(anisotropic_scenarios), 30*np.ones(len(anisotropic_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks([])
    ax.set_yticks([])
    
    ax = plt.subplot(325)
    ax.set_xlabel('\nAnisotropy ratio (log)', size = 10)
    ax.set_ylabel('Flow error', size = 10)
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,0,2], marker = 'o', linestyle = '-') # LC - SCB
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,1,2], marker = 'o', linestyle = '-') # FC - SCB
    ax.plot(np.log10(anisotropic_scenarios), np.zeros(len(anisotropic_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks(np.log10(anisotropic_scenarios))
    ax.legend(['SCB', 'XT3D', 'Analytical'], loc = 'lower right')
    
    ax = plt.subplot(326)
    ax.set_xlabel('\nAnisotropy ratio (log)', size = 10)
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,2,2], marker = 'o', linestyle = '-') # LC - XT3D
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,3,2], marker = 'o', linestyle = '-') # FC - XT3D
    ax.plot(np.log10(anisotropic_scenarios), np.zeros(len(anisotropic_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks(np.log10(anisotropic_scenarios))
    ax.set_yticks([])
    ax.legend(['SCB', 'XT3D', 'Analytical'], loc = 'lower right')

    plt.tight_layout()
    plt.show()
    fig.savefig('../figures/fig5paper.png', dpi = 450)

In [4]:
def table1():
    for i in range(4):
        print(modelnames[i])
        print('qmagmid = ', q_all[i][0])
        print('qangmid = ', q_all[i][1])
        print('vol_flow = ', flow_all[i], ' Analytical =', flow_analytical, '\n')
        #print ("\nSPECIFIC DISCHARGE:")
        #qdata = [
        #    ["analytical", qmagana, qangana],
        #    ["mid-channel", qmagmid, qangmid],
        #    ["avg channel", qmagavg, qangavg],
        #    ["min channel", qmagmin, qangmin],
        #    ["max channel", qmagmax, qangmax],
        #]
        #qhead = ["", "magnitude", "angle",]
        #print(tabulate(qdata, headers=qhead, tablefmt="simple", floatfmt=".14f"))